O processo atualiza arquivos CSV em um diretório, adicionando a coluna data_carga com a data de modificação do arquivo. Ele evita sobrescritas desnecessárias ao verificar se a coluna já existe.

Um relatório chamado cargas_exec.csv é gerado, detalhando o status de cada arquivo (Atualizado, Já Processado ou Erro), a data de modificação e a data de carga.

Arquivos: C:\Users\Admin\Desktop\CQD\cargas
Relatório: C:\Users\Admin\Desktop\CQD\relatorio\cargas_exec.csv

Atualizador de Data de Carga CSV

In [ ]:
# Nome: Importar bibliotecas
import os
import pandas as pd
from datetime import datetime

Definir diretórios e inicializar variáveis

In [ ]:
# Nome: Configurações iniciais
# Diretório principal contendo os arquivos
diretorio_principal = r'C:\Users\Admin\Desktop\CQD\cargas'

# Local para salvar o relatório
diretorio_relatorio = r'C:\Users\Admin\Desktop\CQD\relatorio'
nome_relatorio = 'cargas_exec.csv'

# Lista para armazenar informações dos arquivos processados
relatorio_dados = []

Percorrer arquivos e processar CSVs

In [ ]:
# Nome: Processar arquivos CSV
# Percorre todas as subpastas e arquivos no diretório
for subdir, _, files in os.walk(diretorio_principal):
    for file in files:
        # Caminho completo do arquivo
        file_path = os.path.join(subdir, file)
        
        # Verifica se o arquivo é CSV
        if file.endswith('.csv'):
            try:
                # Carregar o arquivo CSV
                df = pd.read_csv(file_path)
                
                # Verifica se a coluna 'data_carga' já existe
                if 'data_carga' not in df.columns:
                    # Data de modificação do arquivo
                    data_modificacao = datetime.fromtimestamp(os.path.getmtime(file_path)).strftime('%Y-%m-%d %H:%M:%S')
                    
                    # Adicionar a coluna 'data_carga'
                    df['data_carga'] = data_modificacao
                    
                    # Salvar o arquivo atualizado
                    df.to_csv(file_path, index=False)
                    
                    # Adicionar informações ao relatório
                    relatorio_dados.append({
                        'Arquivo': file_path,
                        'Status': 'Atualizado',
                        'Data de Modificação': data_modificacao,
                        'Data de Carga': data_modificacao
                    })
                    print(f"Atualizado: {file_path}")
                else:
                    # Se a coluna já existir, captura o valor de 'data_carga'
                    data_carga = df['data_carga'].iloc[0] if not df['data_carga'].isnull().all() else 'N/A'
                    
                    relatorio_dados.append({
                        'Arquivo': file_path,
                        'Status': 'Já Processado',
                        'Data de Modificação': 'N/A',
                        'Data de Carga': data_carga
                    })
                    print(f"A coluna 'data_carga' já existe: {file_path}. Nenhuma alteração foi feita.")
            except Exception as e:
                # Adicionar informações de erro ao relatório
                relatorio_dados.append({
                    'Arquivo': file_path,
                    'Status': f"Erro: {e}",
                    'Data de Modificação': 'N/A',
                    'Data de Carga': 'N/A'
                })
                print(f"Erro ao processar {file_path}: {e}")

Criar e salvar o relatório

In [ ]:
# Nome: Criar e salvar relatório
# Cria o DataFrame do relatório
relatorio_df = pd.DataFrame(relatorio_dados)

# Garante que o diretório para o relatório exista
os.makedirs(diretorio_relatorio, exist_ok=True)

# Caminho completo do relatório
relatorio_path = os.path.join(diretorio_relatorio, nome_relatorio)

# Salva o relatório em CSV
relatorio_df.to_csv(relatorio_path, index=False)
print(f"Relatório salvo em: {relatorio_path}")


Descrição:
Este script automatiza o processo de mapeamento das colunas de todos os arquivos CSV localizados em um diretório e suas subpastas. Ele gera um relatório consolidado que contém informações sobre o número de colunas e seus respectivos nomes para cada arquivo.

Como Funciona:

Percorre todos os arquivos CSV no diretório especificado e suas subpastas.
Extrai os nomes das colunas de cada arquivo.
Adiciona os dados de mapeamento a uma lista.
Gera um relatório consolidado em formato CSV.
Campos do Relatório:

Arquivo: Caminho completo do arquivo CSV.
Quantidade de Colunas: Número total de colunas no arquivo.
Colunas: Lista dos nomes das colunas, separados por vírgula.
Local de Salvar o Relatório:
O relatório será salvo em: C:\Users\Admin\Desktop\CQD\relatorio\mapeamento_colunas.csv.

Instruções de Uso:

Configure o diretório principal (diretorio_principal) e o local do relatório (diretorio_relatorio).
Execute as células do script em sequência no Jupyter Notebook.
Verifique o relatório gerado no diretório configurado.

In [ ]:
# Nome: Importar Bibliotecas
import os
import pandas as pd


Definir Diretórios

In [ ]:
# Nome: Definir Diretórios
# Diretório principal contendo os arquivos CSV
diretorio_principal = r'C:\Users\Admin\Desktop\CQD\cargas'

# Local para salvar o relatório
diretorio_relatorio = r'C:\Users\Admin\Desktop\CQD\relatorio'
nome_relatorio = 'mapeamento_colunas.csv'


In [ ]:
# Nome: Carregar Relatório Existente
# Verifica se o relatório já existe e carrega os arquivos já verificados
caminho_relatorio = os.path.join(diretorio_relatorio, nome_relatorio)

if os.path.exists(caminho_relatorio):
    df_relatorio_existente = pd.read_csv(caminho_relatorio)
    arquivos_verificados = set(df_relatorio_existente['Arquivo'].tolist())
    print(f"Relatório existente carregado. {len(arquivos_verificados)} arquivos ja verificados.")
else:
    df_relatorio_existente = pd.DataFrame(columns=['Arquivo', 'Quantidade de Colunas', 'Colunas'])
    arquivos_verificados = set()
    print("Nenhum relatório existente encontrado. Todos os arquivos serão verificados.")


In [ ]:
# Nome: Inicializar Lista de Resultados
# Lista para armazenar os resultados do mapeamento
mapeamento = []


In [ ]:
# Nome: Mapear Colunas dos Arquivos CSV
# Percorre todas as subpastas e arquivos no diretório
for subdir, _, files in os.walk(diretorio_principal):
    for file in files:
        # Caminho completo do arquivo
        file_path = os.path.join(subdir, file)
        
        # Verifica se o arquivo já foi processado
        if file_path in arquivos_verificados:
            print(f"Arquivo já verificado: {file_path}. Pulando...")
            continue
        
        # Verifica se o arquivo é CSV
        if file.endswith('.csv'):
            try:
                # Carrega o CSV e captura as colunas
                df = pd.read_csv(file_path)
                colunas = list(df.columns)
                
                # Adiciona ao mapeamento
                mapeamento.append({
                    'Arquivo': file_path,
                    'Quantidade de Colunas': len(colunas),
                    'Colunas': ', '.join(colunas)
                })
                print(f"Arquivo mapeado: {file_path}")
            except Exception as e:
                # Adiciona arquivos com erro ao mapeamento
                mapeamento.append({
                    'Arquivo': file_path,
                    'Quantidade de Colunas': 'Erro',
                    'Colunas': f"Erro: {e}"
                })
                print(f"Erro ao processar {file_path}: {e}")


In [ ]:
# Nome: Atualizar e Salvar Relatório
# Cria um DataFrame com o novo mapeamento
df_mapeamento = pd.DataFrame(mapeamento)

# Combina o relatório existente com os novos resultados
df_relatorio_atualizado = pd.concat([df_relatorio_existente, df_mapeamento], ignore_index=True)

# Garante que o diretório para salvar o relatório exista
os.makedirs(diretorio_relatorio, exist_ok=True)

# Salva o relatório atualizado em um arquivo CSV
df_relatorio_atualizado.to_csv(caminho_relatorio, index=False)
print(f"Relatório atualizado salvo em: {caminho_relatorio}")


Script de Ajuste e Padronização de Arquivos CSV

Processa arquivos CSV em um diretório e realiza ajustes para padronizar os nomes das colunas, verificar colunas ausentes ou excedentes, e corrigir valores específicos. Além disso, gera um relatório detalhado com informações sobre os ajustes realizados.

Padronização de Colunas:

Converte os nomes de todas as colunas para maiúsculas e remove acentos.
Renomeia colunas específicas, como DADOS_SENSÍVEIS para DADOS_SENSIVEIS.
Preenchimento e Correção de Dados:

Preenche valores ausentes em IP_SERVIDOR com os da coluna IP, caso existam.
Remove colunas não esperadas ou extras, como IP após o ajuste.
Relatório Detalhado:

Lista de colunas ausentes e excedentes por arquivo.
Número de colunas antes e após os ajustes.
Coluna ajustada, se aplicável.
Status de cada arquivo (Ajustado ou Erro).
Incrementalidade:

Não processa novamente arquivos que já foram ajustados, com base em um relatório existente.


In [ ]:
# Nome: Importação de Bibliotecas
import os
import pandas as pd
import unicodedata


Definição de Diretórios e Padrão de Colunas

In [ ]:
# Nome: Definição de Diretórios e Padrão de Colunas

# Diretório principal contendo os arquivos CSV
diretorio_principal = r'C:\Users\Admin\Desktop\CQD\cargas'

# Local para salvar o relatório
diretorio_relatorio = r'C:\Users\Admin\Desktop\CQD\relatorio'
nome_relatorio = 'ajuste_padronizacao.csv'

# Padrão esperado de colunas
colunas_padrao = [
    "NOME_DO_SGBD", "IP_SERVIDOR", "VERSAO_DO_SGDB", "NOME_BANCO", "NOME_SCHEMA",
    "NOME_TABELA", "TIPO_CARACTERISTICA", "NOME_COLUNA", "TIPO_DADO", "PERMITE_NULO",
    "TIPO_DE_CHAVE", "TIPO_DE_CAMPO", "DADOS_SENSIVEIS", "DATA_CARGA"
]

# Função para remover acentos
def remover_acentos(string):
    if not isinstance(string, str):
        return string
    return ''.join(c for c in unicodedata.normalize('NFD', string) if unicodedata.category(c) != 'Mn')


In [ ]:
# Nome: Carregar Relatório Anterior

# Carrega o relatório anterior, se existir
relatorio_path = os.path.join(diretorio_relatorio, nome_relatorio)
if os.path.exists(relatorio_path):
    relatorio_anterior = pd.read_csv(relatorio_path)
    arquivos_ajustados = set(relatorio_anterior['Arquivo'].tolist())
    print(f"Relatório existente carregado. {len(arquivos_ajustados)} arquivos já ajustados.")
else:
    relatorio_anterior = pd.DataFrame(columns=['Arquivo', 'Status', 'Colunas Faltantes', 'Colunas a Mais', 'Coluna Ajustada', 'Colunas Antes', 'Colunas Depois'])
    arquivos_ajustados = set()
    print("Nenhum relatório existente encontrado. Todos os arquivos serão processados.")


In [ ]:
# Nome: Inicializar Relatório Novo

# Lista para o novo relatório
relatorio_dados = []


In [ ]:
# Nome: Processar Arquivos CSV

# Processa cada arquivo CSV no diretório
for subdir, _, files in os.walk(diretorio_principal):
    for file in files:
        file_path = os.path.join(subdir, file)
        
        # Verifica se o arquivo já foi ajustado
        if file_path in arquivos_ajustados:
            print(f"Arquivo já ajustado: {file_path}. Pulando...")
            continue

        if file.endswith('.csv'):
            try:
                # Carregar o arquivo
                df = pd.read_csv(file_path)
                colunas_antes = len(df.columns)  # Número de colunas antes do ajuste
                colunas_atuais = df.columns.tolist()
                coluna_ajustada = None

                # Identifica colunas a mais (não estão no padrão esperado)
                colunas_a_mais = [col for col in colunas_atuais if remover_acentos(col.upper()) not in colunas_padrao]

                # Padroniza os nomes das colunas
                df.rename(columns=lambda x: remover_acentos(x).upper(), inplace=True)
                
                # Renomeia "DADOS_SENSÍVEIS" para "DADOS_SENSIVEIS"
                if "DADOS_SENSÍVEIS" in df.columns:
                    df.rename(columns={"DADOS_SENSÍVEIS": "DADOS_SENSIVEIS"}, inplace=True)
                    coluna_ajustada = "DADOS_SENSIVEIS"
                
                # Preenche IP_SERVIDOR com valores de IP, se necessário
                if "IP_SERVIDOR" in df.columns and "IP" in df.columns:
                    df["IP_SERVIDOR"] = df["IP_SERVIDOR"].fillna(df["IP"])
                    coluna_ajustada = "IP_SERVIDOR"
                
                # Remove a coluna "IP"
                if "IP" in df.columns:
                    df.drop(columns=["IP"], inplace=True)

                # Remove colunas extras ou não nomeadas
                df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

                # Adiciona colunas faltantes
                for col in colunas_padrao:
                    if col not in df.columns:
                        df[col] = None

                # Ordena as colunas no padrão esperado
                df = df[colunas_padrao]
                colunas_depois = len(df.columns)  # Número de colunas após o ajuste

                # Salva o arquivo ajustado
                df.to_csv(file_path, index=False)
                
                # Atualiza o relatório
                relatorio_dados.append({
                    "Arquivo": file_path,
                    "Status": "Ajustado",
                    "Colunas Faltantes": ", ".join(set(colunas_padrao) - set(colunas_atuais)),
                    "Colunas a Mais": ", ".join(colunas_a_mais),
                    "Coluna Ajustada": coluna_ajustada if coluna_ajustada else "Nenhuma",
                    "Colunas Antes": colunas_antes,
                    "Colunas Depois": colunas_depois
                })
                print(f"Ajustado: {file_path}")
            
            except Exception as e:
                # Atualiza o relatório com erros
                relatorio_dados.append({
                    "Arquivo": file_path,
                    "Status": f"Erro: {e}",
                    "Colunas Faltantes": "N/A",
                    "Colunas a Mais": "N/A",
                    "Coluna Ajustada": "N/A",
                    "Colunas Antes": "N/A",
                    "Colunas Depois": "N/A"
                })
                print(f"Erro ao processar {file_path}: {e}")


In [ ]:
# Nome: Salvar Relatório Atualizado

# Combina o novo relatório com o existente
relatorio_atualizado = pd.concat([relatorio_anterior, pd.DataFrame(relatorio_dados)], ignore_index=True)

# Garante que o diretório para salvar o relatório exista
os.makedirs(diretorio_relatorio, exist_ok=True)

# Salva o relatório atualizado
relatorio_atualizado.to_csv(relatorio_path, index=False)

print(f"Relatório atualizado salvo em: {relatorio_path}")
